In [ ]:
from glob import glob
import os
import mne
from scipy.io import loadmat
from scipy import signal,stats
import pandas as pd
import numpy as np
from numpy.fft import fft , ifft
import matplotlib.pyplot as plt 
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs,corrmap)
import re

In [ ]:
path_os = os.getcwd()
PATH = os.path.join(path_os,'data','v1')

'''
TOTAL_PATH_T_1_3 = [
    [[s1],[s2],...,[s6]], ---->T1
    [[s1],[s2],...,[s6]], ---->T2
    [[s1],[s2],...,[s6]], ---->T3
]
'''

TOTAL_PATH_T_1_3 = []



#مقدار فور را برای ۳ تحریک تنظیم می‌کنیم.
for T in range(1,4):
    TOTAL_PATH_SUBJECT_1_6_T = []
    #مقدار فور را برای ۶ سابجکت تنظیم کرده ایم
    for subject in range(1,7):
        """
            در اینجا اومدیم نحوه گرفتن داده ها برای تحریک های مخنلف را جدا کردیم. برای گرفتن داده ی سابجکت ۱ تا ۶ برای تحریک شماره ۱
            /Directions_and_Time_T1/*.mat
            برای گرفتن داده های سابجکت شماره ۱تا ۶ تحریک شماره ۲
            /Directions_and_Time_T2/*.mat
            برای گرفتن داده های سابجکت ۱تا ۶ برای تحریک شماره ۳
            /Directions_and_Time_T3/*.mat
        """ 
        subject_T = glob(PATH+f'/s{subject}/Directions_and_Time_T{T}/*.mat')
        TOTAL_PATH_SUBJECT_1_6_T.append(subject_T)
    
    TOTAL_PATH_T_1_3.append(TOTAL_PATH_SUBJECT_1_6_T)







In [ ]:
#preprocessing data without ICA

map_label = [
        {'shoroT1':0,'balaT1':1,'paeinT1':2,'raastT1':3,'chapT1':4,'jeloT1':5,'aqabT1':6,'payanT1':7},
        {'shoroT2':0,'balaT2':1,'paeinT2':2,'raastT2':3,'chapT2':4,'jeloT2':5,'aqabT2':6,'payanT2':7},
        {'shoroT3':0,'balaT3':1,'paeinT3':2,'raastT3':3,'chapT3':4,'jeloT3':5,'aqabT3':6,'payanT3':7},          
]

ch_names = ['Fpz','Fp1','Fp2','Fz','F3','F4','F7','F8','Cz','C3','C4','T3','T4','Pz','P3','P4','T5','T6','O1', 'O2','Oz']
ch_types = ['eeg'] * 21
sampling_freq = 2000
info = mne.create_info(ch_names, ch_types=ch_types, sfreq=sampling_freq)
info.set_montage('standard_1020')


frequency_band = np.array[[0.1,4,8,12,30],[4,8,12,30,70]]

def preProcessingEegSingal(list_path_subect_i,subject,PATH_SAVE,T_Couner):
    '''
        
        
این جا همه ی مسیرهای مربروط به هر سابجکت را به عنوان ورودی تابع می‌گیریم و تمام دیتای موجود برای سابجکت۱ و لیبل های آن را درون دوتا لیست ذخیره می‌کنیم.
آرگومان اول تابع مسیر داده ها می‌باشد که برای سابجکت های مختلف از طریق یک فور وارد تابع می شه.
آرگومان دوم شماره سابجکت هستش که می تونه مقادیر ۱ تا ۶ را داشته باشد
آرگومان سوم مسیر ذخیره سازی داده ی تمیز را برا تابع می‌فرستیم که در این مسیر دیتا ذخیره بشه.

    '''

    #وقتی دیتا با فرمت مت را میخونیم یک دیکشنری به ما می‌دهد که باید دیتا و لیبل ها را را جدا کنیم.
    data_for_subject_t = []
    label_for_subject_t1 = []
    label_mapping_to_intiger = []

    #این فور برای گرفتن تک تک داده های سابجکت مثلا شماره یک است که هر بار آدرس تمام داده های هر سابجکت وارد تابع می‌شه و با این فور به هر کدام از آدرس ها دسترسی پیدا می‌کنیم
    for path_mat in list_path_subect_i:
        
        Temp = loadmat(path_mat)
        data_for_subject_t.append(7e-7*Temp['EEG_Data'])
        re_for_get_label=re.findall(r'ID.*event_id',str(Temp['Labels'][0]))[0].split(",")[0]
        
        re_for_get_label=re_for_get_label.split(':')[1].strip()
        
        label_for_subject_t1.append(re_for_get_label.replace("'",''))
        
        if 'Directions_and_Time_T1' in PATH_SAVE:
            label_mapping_to_intiger.append(map_label[0].get(label_for_subject_t1[-1]))
        elif 'Directions_and_Time_T2' in PATH_SAVE:
            label_mapping_to_intiger.append(map_label[1].get(label_for_subject_t1[-1]))
        elif 'Directions_and_Time_T3' in PATH_SAVE:
            label_mapping_to_intiger.append(map_label[2].get(label_for_subject_t1[-1]))

    
    
    DATA_RAW = []
    DATA_RAW_DOWN = []
    
    for data_for_clean in range(len(data_for_subject_t)):
        raw = mne.io.RawArray(data_for_subject_t[data_for_clean][:,:], info)
        raw.crop(tmax=1.5) 
        DATA_RAW.append(raw['data'][0])
        raw.plot(show_scrollbars=False, show_scalebars=False)
        filt_raw=raw.filter(l_freq=1, h_freq=40)
        filt_raw.plot(start=0,show_scrollbars=False, show_scalebars=False)
        
        #raw_down.plot(start=0,show_scrollbars=False, show_scalebars=False)
        DATA_RAW_DOWN.append(raw_down['data'][0])
    
    
    DATA_RAW_DOWN = np.array(DATA_RAW_DOWN)
    label_mapping_to_intiger=np.array(label_mapping_to_intiger)
    label_for_subject_t1=np.array(label_for_subject_t1)
    
    #np.save(PATH_SAVE+f'Data_CLEAN_s{subject}_T{T_Couner}',DATA_RAW_DOWN) #save clean data
    #np.save(PATH_SAVE+f'LABEL_subject_{subject}_symbol',label_for_subject_t1) # save label of each sample (balaT1,payanT1,...)
    #np.save(PATH_SAVE+f'LABEL_subject_{subject}_intiger',label_mapping_to_intiger) #save label of each sample mapping to init (0,1,2,3)
    
    return DATA_RAW_DOWN,label_for_subject_t1,label_mapping_to_intiger

    
T_Couner = 1 
for path_for_t in TOTAL_PATH_T_1_3:
    for i in range(len(path_for_t)):
        data,label_name_symbole,label_init =preProcessingEegSingal(path_for_t[i],i+1,PATH+f'/s{i+1}/Directions_and_Time_T{T_Couner}/preprocessing_data/',T_Couner)
    T_Couner+=1  




In [ ]:
#preprocessing data with ICA
map_label = [
        {'shoroT1':0,'balaT1':1,'paeinT1':2,'raastT1':3,'chapT1':4,'jeloT1':5,'aqabT1':6,'payanT1':7},
        {'shoroT2':0,'balaT2':1,'paeinT2':2,'raastT2':3,'chapT2':4,'jeloT2':5,'aqabT2':6,'payanT2':7},
        {'shoroT3':0,'balaT3':1,'paeinT3':2,'raastT3':3,'chapT3':4,'jeloT3':5,'aqabT3':6,'payanT3':7},          
]

ch_names = ['Fpz','Fp1','Fp2','Fz','F3','F4','F7','F8','Cz','C3','C4','T3','T4','Pz','P3','P4','T5','T6','O1', 'O2','Oz']
ch_types = ['eeg'] * 21
sampling_freq = 2000
info = mne.create_info(ch_names, ch_types=ch_types, sfreq=sampling_freq)
info.set_montage('standard_1020')

    

def preProcessingEegSingal(list_path_subect_i,subject,PATH_SAVE,T_Couner):
    '''
        
        
این جا همه ی مسیرهای مربروط به هر سابجکت را به عنوان ورودی تابع می‌گیریم و تمام دیتای موجود برای سابجکت۱ و لیبل های آن را درون دوتا لیست ذخیره می‌کنیم.
آرگومان اول تابع مسیر داده ها می‌باشد که برای سابجکت های مختلف از طریق یک فور وارد تابع می شه.
آرگومان دوم شماره سابجکت هستش که می تونه مقادیر ۱ تا ۶ را داشته باشد
آرگومان سوم مسیر ذخیره سازی داده ی تمیز را برا تابع می‌فرستیم که در این مسیر دیتا ذخیره بشه.

    '''

    #وقتی دیتا با فرمت مت را میخونیم یک دیکشنری به ما می‌دهد که باید دیتا و لیبل ها را را جدا کنیم.
    data_for_subject_t = []

    #این فور برای گرفتن تک تک داده های سابجکت مثلا شماره یک است که هر بار آدرس تمام داده های هر سابجکت وارد تابع می‌شه و با این فور به هر کدام از آدرس ها دسترسی پیدا می‌کنیم
    for path_mat in list_path_subect_i:
        
        Temp = loadmat(path_mat)
        data_for_subject_t.append(7e-7*Temp['EEG_Data'])

    
    
    DATA_RAW = []
    DATA_RAW_DOWN = []
    

    for data_for_clean in range(len(data_for_subject_t)):
        raw = mne.io.RawArray(data_for_subject_t[data_for_clean][:,:], info)
        raw.crop(tmax=1.5) 
        
        filt_raw = raw.filter(l_freq=1, h_freq=40)
        
        
        ica = ICA(n_components=15, max_iter='auto', random_state=97)
        ica.fit(filt_raw)
        ica.apply(filt_raw)

        raw_down = filt_raw.resample(sfreq=80)
        #raw_down.plot(start=0,show_scrollbars=False, show_scalebars=False)
        DATA_RAW_DOWN.append(raw_down['data'][0])
        
    DATA_RAW_DOWN = np.array(DATA_RAW_DOWN)
    
    np.save(PATH_SAVE+f'Data_CLEAN_ICA_s{subject}_T{T_Couner}',DATA_RAW_DOWN)
    
    

    
T_Couner = 1 
for path_for_t in TOTAL_PATH_T_1_3:
    for i in range(len(path_for_t)):
        preProcessingEegSingal(path_for_t[i],i+1,PATH+f'/s{i+1}/Directions_and_Time_T{T_Couner}/preprocessing_data/',T_Couner)
    T_Couner+=1



In [ ]:
s_1 = TOTAL_PATH_T_1_3[1][0]
Temp = loadmat(s_1[20])
data = 7e-7*Temp['EEG_Data']
data = data.transpose()
re_for_get_label=re.findall(r'ID.*event_id',str(Temp['Labels'][0]))[0].split(",")[0]

re_for_get_label=re_for_get_label.split(':')[1].strip()

label = re_for_get_label.replace("'",'')
N = len(data[:,0])
fs = 2000
rf = np.linspace(0,fs/4,int(np.round(N/2)))

fft_data = fft(data[:,0],n = N)

In [ ]:
plt.stem(rf[1:],np.abs(fft_data[1:int(np.round(N/2))]*10))
plt.show()

In [ ]:
#preprocessing data with ICA
map_label = [
        {'shoroT1':0,'balaT1':1,'paeinT1':2,'raastT1':3,'chapT1':4,'jeloT1':5,'aqabT1':6,'payanT1':7},
        {'shoroT2':0,'balaT2':1,'paeinT2':2,'raastT2':3,'chapT2':4,'jeloT2':5,'aqabT2':6,'payanT2':7},
        {'shoroT3':0,'balaT3':1,'paeinT3':2,'raastT3':3,'chapT3':4,'jeloT3':5,'aqabT3':6,'payanT3':7},          
]
s_1 = TOTAL_PATH_T_1_3[1][3]
Temp = loadmat(s_1[15])
data = Temp['EEG_Data']

ch_names = ['Fpz','Fp1','Fp2','Fz','F3','F4','F7','F8','Cz','C3','C4','T3','T4','Pz','P3','P4','T5','T6','O1', 'O2','Oz']
ch_types = ['eeg'] * 21 #برای اینکه بفهمد که همه کانال ها سیگنال (ای ای جی) می‌گیرد

sampling_freq = 2000

info = mne.create_info(ch_names, ch_types=ch_types, sfreq=sampling_freq)
info.set_montage('standard_1020')

raw = mne.io.RawArray(data, info)
raw.crop(tmax=1.8)
filt_raw = raw.filter(l_freq=1, h_freq=40)
filt_raw.resample(sfreq=sampling_freq) #بعد از کراپ کردن دیتا دوباره می‌ایم یک نمونه برداری می‌کنیم که سایز ماتریس داده ها برای همه دیتا ها یکی شود
filt_raw['data'][0].shape


In [ ]:
s_1 = TOTAL_PATH_T_1_3[0][1]
Temp = loadmat(s_1[10])
a = 7e-7
data = a * Temp['EEG_Data']
chanel_num = data.shape[0]
sample_for_data = data.shape[1]
data_filter = np.zeros_like(data)
fourier_domain = np.zeros_like(data)

In [ ]:
#get just one data
#denoising for one signal
s_1 = TOTAL_PATH_T_1_3[0][2]
Temp = loadmat(s_1[10])
a = 7e-7
data = a * Temp['EEG_Data']
re_for_get_label=re.findall(r'ID.*event_id',str(Temp['Labels'][0]))[0].split(",")[0]

re_for_get_label=re_for_get_label.split(':')[1].strip()

label_for_subject_t = re_for_get_label.replace("'",'')
chanel_num = data.shape[0]
sample_for_data = data.shape[1]
data_filter = np.zeros_like(data)
fourier_domain = np.zeros_like(data)
fs = 2000
f_nyq = fs/2
Wn = [1,40]/np.float_(f_nyq)
(b,a) = signal.butter(4, Wn, btype='bandpass', output='ba')
rf = np.linspace(0,f_nyq,int(np.round(sample_for_data/2)))
rf = rf[rf<50]
for i in range(0,chanel_num):
    #چون ۲۱ کانال داریم باید هر بار یک کانال را بگیریم بعد آ ن را دینویز کنیم بهد ازش فیلتر بگیریم.
    sigA = data[i,:]
    data_filter[i,:] = signal.filtfilt(b,a,sigA)
    print(signal.filtfilt(b,a,sigA).shape)
    Fsig_dn = fft(data_filter[i,:],sample_for_data)
    fourier_domain[i,:] = Fsig_dn
    
    # plt.figure(figsize=(12,8),dpi = 300)
    # plt.subplot(1,2,1)
    # plt.stem(rf,np.abs(fourier_domain[i,:rf.shape[0]]),label=f'channel {i+1} - frequency domain')
    # plt.legend()
    # plt.subplot(1,2,2)
    # plt.plot(data_filter[i,:],label=f'channel {i+1}- time domain ({label_for_subject_t})')
    # plt.legend()
    # plt.savefig(f'{i}.png')


In [ ]:
3997

In [ ]:
plt.figure(figsize=(12,8),dpi = 300)
plt.subplot(1,2,1)
plt.stem(rf,np.abs(fourier_domain[2,:rf.shape[0]]),label=f'channel {i+1}- frequency domain')
plt.legend()
plt.subplot(1,2,2)
plt.plot(data_filter[2,:],label=f'channel {i+1}- time domain')
plt.legend()
plt.savefig(f'{i}.png')

In [ ]:
T_Couner = 1 
MIN = 5000
MAX = 0
for path_for_t in TOTAL_PATH_T_1_3:
    for i in range(len(path_for_t)):
        for j in path_for_t[i]:
            Temp = loadmat(j)
            data = Temp['EEG_Data']
            
            a,b = data.shape
            if b < MIN:
                MIN = b
            if b > MAX :
                MAX = b




In [ ]:
map_label = [
        {'shoroT1':0,'balaT1':1,'paeinT1':2,'raastT1':3,'chapT1':4,'jeloT1':5,'aqabT1':6,'payanT1':7},
        {'shoroT2':0,'balaT2':1,'paeinT2':2,'raastT2':3,'chapT2':4,'jeloT2':5,'aqabT2':6,'payanT2':7},
        {'shoroT3':0,'balaT3':1,'paeinT3':2,'raastT3':3,'chapT3':4,'jeloT3':5,'aqabT3':6,'payanT3':7},          
]

def preProcessingEegSingal(list_path_subect_i,subject,PATH_SAVE,T_Couner):
    '''
        
        
این جا همه ی مسیرهای مربروط به هر سابجکت را به عنوان ورودی تابع می‌گیریم و تمام دیتای موجود برای سابجکت۱ و لیبل های آن را درون دوتا لیست ذخیره می‌کنیم.
آرگومان اول تابع مسیر داده ها می‌باشد که برای سابجکت های مختلف از طریق یک فور وارد تابع می شه.
آرگومان دوم شماره سابجکت هستش که می تونه مقادیر ۱ تا ۶ را داشته باشد
آرگومان سوم مسیر ذخیره سازی داده ی تمیز را برا تابع می‌فرستیم که در این مسیر دیتا ذخیره بشه.

    '''

    #وقتی دیتا با فرمت مت را میخونیم یک دیکشنری به ما می‌دهد که باید دیتا و لیبل ها را را جدا کنیم.
    data_for_subject_t = []
    label_for_subject_t1 = []
    label_mapping_to_intiger = []

    fs = 2000
    f_nyq = fs/2
    Wn = [1,40]/np.float_(f_nyq)
    (b,a) = signal.butter(4, Wn, btype='bandpass', output='ba')

    #این فور برای گرفتن تک تک داده های سابجکت مثلا شماره یک است که هر بار آدرس تمام داده های هر سابجکت وارد تابع می‌شه و با این فور به هر کدام از آدرس ها دسترسی پیدا می‌کنیم
    for path_mat in list_path_subect_i:
        
        Temp = loadmat(path_mat)
        
        a = 7e-7
        
        data = a * Temp['EEG_Data']
        
        re_for_get_label=re.findall(r'ID.*event_id',str(Temp['Labels'][0]))[0].split(",")[0]
        
        re_for_get_label = re_for_get_label.split(':')[1].strip()
        
        label_for_subject_t = re_for_get_label.replace("'",'')
        
        chanel_num = data.shape[0]
        sample_for_data = data.shape[1]
        
        data_filter = np.zeros_like(data)
        fourier_domain = np.zeros_like(data)
        
        if 'Directions_and_Time_T1' in PATH_SAVE:
            label_mapping_to_intiger.append(map_label[0].get(label_for_subject_t1[-1]))
        elif 'Directions_and_Time_T2' in PATH_SAVE:
            label_mapping_to_intiger.append(map_label[1].get(label_for_subject_t1[-1]))
        elif 'Directions_and_Time_T3' in PATH_SAVE:
            label_mapping_to_intiger.append(map_label[2].get(label_for_subject_t1[-1]))

    
    

    rf = np.linspace(0,f_nyq,int(np.round(sample_for_data/2)))
    rf = rf[rf<50]
    
    for i in range(0,chanel_num):
        #چون ۲۱ کانال داریم باید هر بار یک کانال را بگیریم بعد آ ن را دینویز کنیم بهد ازش فیلتر بگیریم.
        sigA = data[i,:]
        data_filter[i,:] = signal.filtfilt(b,a,sigA)
        
        Fsig_dn = fft(data_filter[i,:],sample_for_data)
        
        fourier_domain[i,:] = Fsig_dn
        
        # plt.figure(figsize=(12,8),dpi = 300)
        # plt.subplot(1,2,1)
        # plt.stem(rf,np.abs(fourier_domain[i,:rf.shape[0]]),label=f'channel {i+1} - frequency domain')
        # plt.legend()
        # plt.subplot(1,2,2)
        # plt.plot(data_filter[i,:],label=f'channel {i+1}- time domain ({label_for_subject_t})')
        # plt.legend()
        # plt.savefig(f'{i}.png')
        
    
    DATA_RAW_DOWN = np.array(DATA_RAW_DOWN)
    label_mapping_to_intiger = np.array(label_mapping_to_intiger)
    label_for_subject_t1 = np.array(label_for_subject_t1)
    
    #np.save(PATH_SAVE+f'Data_CLEAN_s{subject}_T{T_Couner}',DATA_RAW_DOWN) #save clean data
    #np.save(PATH_SAVE+f'LABEL_subject_{subject}_symbol',label_for_subject_t1) # save label of each sample (balaT1,payanT1,...)
    #np.save(PATH_SAVE+f'LABEL_subject_{subject}_intiger',label_mapping_to_intiger) #save label of each sample mapping to init (0,1,2,3)
    
    return DATA_RAW_DOWN,label_for_subject_t1,label_mapping_to_intiger

    
T_Couner = 1 
for path_for_t in TOTAL_PATH_T_1_3:
    for i in range(len(path_for_t)):
        data,label_name_symbole,label_init =preProcessingEegSingal(path_for_t[i],i+1,PATH+f'/s{i+1}/Directions_and_Time_T{T_Couner}/preprocessing_data/',T_Couner)
    T_Couner+=1  




In [ ]:
path = path_os = os.getcwd()
PATH = os.path.join(path_os,'data','v1')
subject_T = glob(PATH+f'/s{1}/Directions_and_Time_T{1}/*.mat')
data_raw_path = subject_T[0]

def filter_noisy_data(data,channel_num,sample_for_data,fs,f_nyq,f_low,f_high):
    
    data_filter = np.zeros_like(data)

    Wn = [f_low,f_high]/np.float_(f_nyq)
    
    (b,a) = signal.butter(4, Wn, btype='bandpass', output='ba')

    for channel_signal in range(0,channel_num):
        
        data_each_channel = data[channel_signal,:]
        data_filter[channel_signal,:] = signal.filtfilt(b,a,data_each_channel)

    return data_filter
    
def feature_selection(data,channel_num):
    num_feature = 5 #mean var power skew kur 
    feature_selections_total = np.zeros((21,num_feature))
    # 30 why?
    #5 band that each band has 6 features
    #data = delta that 21*n, theta 21*n alpha 21*n, beta 21*n and gamma 21*n 
    low_band_zeros_matrix = 0
    high_band_zeros_matrix = num_feature
    
    for i in range(0,channel_num):
        
        data_for_feature_extraction = data[i,:]
        
        MEAN = np.mean(data_for_feature_extraction)
        VAR = np.var(data_for_feature_extraction)
        POWER = np.mean(np.power(data_for_feature_extraction,2))
        SKEW = stats.skew(data_for_feature_extraction)
        KUR = stats.kurtosis(data_for_feature_extraction)
        
    


        # مثلا کانال ۱ دارای ۵ تا باند هست و هر باند ۶ فیچر استخراج می‌کنیم پس هر کانال ۳۰ قیچر داره
        
        feature_selections_total[i,:] = (MEAN,VAR,POWER,SKEW,KUR)
        
        
        #low_band_zeros_matrix += 5
        #high_band_zeros_matrix +=5

        #پس این خروجی ماتریس مربوط به ۳۰ ویژگی تمام کانال های یک سیگنال است.
    return feature_selections_total
    


def seperate_band(data,channel_num,sample_for_data,fs,f_nyq):
    #یک ماتریس مثلا ۲۱*۳۹۰۰ که همش سیگنال های باند دلتا را دارد
    delta = filter_noisy_data(data,channel_num,sample_for_data,fs,f_nyq,f_low=1,f_high=4)
    
    #یک ماتریس مثلا ۲۱*۳۹۰۰ که همش سیگنال های باند تتا را دارد
    theta = filter_noisy_data(data,channel_num,sample_for_data,fs,f_nyq,f_low=4,f_high=8)
    
    #یک ماتریس مثلا ۲۱*۳۹۰۰ که همش سیگنال های باند آلفا را دارد     
    alpha = filter_noisy_data(data,channel_num,sample_for_data,fs,f_nyq,f_low=8,f_high=12)

    #یک ماتریس مثلا ۲۱*۳۹۰۰ که همش سیگنال های باند بتا را دارد
    beta = filter_noisy_data(data,channel_num,sample_for_data,fs,f_nyq,f_low=12,f_high=30)

    #یک ماتریس مثلا ۲۱*۳۹۰۰ که سیگنال های باند گاما در هر کانال را دارد
    gamma = filter_noisy_data(data,channel_num,sample_for_data,fs,f_nyq,f_low=30,f_high=40)

    return (delta,theta,alpha,beta,gamma)

def frequency_domain(data):
    f_r[idx_delta],np.abs(fft_data[:int(np.round(N/2))][idx_delta])
    idx_delta = np.logical_and(f_r >= 1, f_r <= 4)
    
    frequency_band_delta = 
    idx_theta = np.logical_and(f_r >= 4, f_r <= 8)
    idx_alpha = np.logical_and(f_r >= 8, f_r <= 12)
    idx_beta = np.logical_and(f_r >= 12, f_r <= 30)
    idx_gamma = np.logical_and(f_r >= 30, f_r <= 40)


def read_raw_data(path):
    
    fs = 2000
    f_nyq = fs/2
    
    
    Temp = loadmat(path)
    a = 7e-7
    data = a * Temp['EEG_Data']
    
    #determine the label for data
    re_for_get_label=re.findall(r'ID.*event_id',str(Temp['Labels'][0]))[0].split(",")[0]
    re_for_get_label=re_for_get_label.split(':')[1].strip()
    label_for_subject_t = re_for_get_label.replace("'",'')

    #determine channel number eeg signal i
    channel_num = data.shape[0]
    #determine sample number for eeg signal i
    sample_for_data = data.shape[1]

    #determine 0 - 40 hz
    data_denoising = filter_noisy_data(data, channel_num, sample_for_data, fs, f_nyq, f_low=1,f_high=40)
    #####################################################seperate frequency band
    (delta_time,theta_time,alpha_time,beta_time,gamma_time) = seperate_band(data_denoising,channel_num,sample_for_data,fs,f_nyq)
    #####################################################Feature Extraction
    delta_feature_time = feature_selection(delta_time,21) #فیچر باند دلتا هر ۲۱ کانال در آن می‌باشد
    theta_feature_time = feature_selection(theta_time,21)#فیچر باند تتا هر ۲۱ کانال در آن می‌باشد
    alpha_feature_time = feature_selection(alpha_time,21)#فیچر باند آلفا هر ۲۱ کانال در آن می‌باشد
    beta_feature_time = feature_selection(beta_time,21)#فیچر باند بتا هر ۲۱ کانال در آن می‌باشد
    gamma_feature_time = feature_selection(gamma_time,21)#فیچر باند گاما هر ۲۱ کانال در آن می‌باشد
    #######################################################
    #میخوایم برای هر کانال تمام فیچر های هر باند را پشت سر هم بندازیم
    num_features_for_each_channel = 25
    time_domain_combine_feature_each_band = np.zeros((channel_num,num_features_for_each_channel))
    
    for chan_num in  range(0,channel_num):
        time_domain_combine_feature_each_band[chan_num,:] = np.ravel(np.vstack((delta_feature_time[chan_num,:],theta_feature_time[chan_num,:],alpha_feature_time[chan_num,:],beta_feature_time[chan_num,:],gamma_feature_time[chan_num,:])))
    
    total_feature_selection_for_each_eeg_signal_time_domain = np.ravel(time_domain_combine_feature_each_band)
    ##################################################################

    return data_denoising
    

In [ ]:
data_denoising = read_raw_data(data_raw_path)
N = data_denoising.shape[1]
fs = 2000
fft_data = fft(data_denoising[1,:],N)
f_r = np.linspace(0,fs/2,int(np.round(N/2)))
f_r_total = np.concatenate((f_r,f_r[::-1]))
fft_data.shape


In [ ]:
for i in range(21):
    plt.stem(f_r[f_r<50],fft_data[i,:f_r[f_r<50].shape[0]])
    plt.show()

In [ ]:
idx_delta = np.logical_and(f_r >= 1, f_r <= 4)
idx_theta = np.logical_and(f_r >= 4, f_r <= 8)
idx_alpha = np.logical_and(f_r >= 8, f_r <= 12)
idx_beta = np.logical_and(f_r >= 12, f_r <= 30)
idx_gamma = np.logical_and(f_r >= 30, f_r <= 40)

In [ ]:
plt.stem(f_r[idx_delta],np.abs(fft_data[:int(np.round(N/2))][idx_delta]),'r')
plt.stem(f_r[idx_theta],np.abs(fft_data[:int(np.round(N/2))][idx_theta]),'g')
plt.stem(f_r[idx_alpha],np.abs(fft_data[:int(np.round(N/2))][idx_alpha]),'y')
plt.stem(f_r[idx_beta],np.abs(fft_data[:int(np.round(N/2))][idx_beta]),'b')
plt.stem(f_r[idx_gamma],np.abs(fft_data[:int(np.round(N/2))][idx_gamma]))
plt.show()

In [ ]:
plt.stem(f_r[f_r<40],np.abs(fft_data[:int(np.round(N/2))][f_r<40]),'r')

In [ ]:
fft_data[:int(np.round(N/2))][idx_gamma]